In [1]:
import sys
from pathlib import Path
import time

# Set manually based on your repo's structure
repo_root = Path.cwd().parent  # or Path.cwd() if you're in the repo root
sys.path.append(str(repo_root))

# LuxendoManager Example Notebook

In [2]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os

In [3]:
# --- Set input data path ---
main_path = "/g/mif/people/gritti/code/pymif_test_data"
dataset_name = "luxendo"

main_path = "/g/mif/common/Data/Z_from_LCS_24-07-16/Coll_Montse/MouseEmbryo_micemetyleneBlue/2022-08-09_132304/processed"
dataset_name = "20221108-162726_Task_2_LM_FullRes_C"

main_path = "/mif-users/Users/Arias_Adrian/scans_CSL_P21/2400_31573_s"
dataset_name = "20250523-105843_Task_1_stitching_1_C"

luxendo_path = f"{main_path}/{dataset_name}"

In [4]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: (1,1,16,256,256)
dataset = mm.LuxendoManager(path=luxendo_path, chunks=(1,1,32,2048,2048))

In [5]:
# --- Show metadata summary ---
print("Axes:", dataset.metadata["axes"])
print("Size:", dataset.metadata["size"])
print("Scales:", dataset.metadata["scales"])
print("Chunk size:", dataset.chunks)
print("Channels:", dataset.metadata["channel_names"])
print("Colors:", dataset.metadata["channel_colors"])
print("\n")
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")

Axes: tczyx
Size: [(1, 2, 4982, 16304, 12055), (1, 2, 2496, 8192, 6080), (1, 2, 1280, 4096, 3072), (1, 2, 640, 2048, 1536), (1, 2, 320, 1024, 768), (1, 2, 192, 512, 384), (1, 2, 128, 256, 192)]
Scales: [(4.0, 1.4625, 1.4625), (8.0, 2.925, 2.925), (16.0, 5.85, 5.85), (32.0, 11.7, 11.7), (64.0, 23.4, 23.4), (128.0, 46.8, 46.8), (256.0, 93.6, 93.6)]
Chunk size: (1, 1, 32, 2048, 2048)
Channels: ['ch:0', 'ch:1']
Colors: [16711680, 255]


SIZE: [(1, 2, 4982, 16304, 12055), (1, 2, 2496, 8192, 6080), (1, 2, 1280, 4096, 3072), (1, 2, 640, 2048, 1536), (1, 2, 320, 1024, 768), (1, 2, 192, 512, 384), (1, 2, 128, 256, 192)]
SCALES: [(4.0, 1.4625, 1.4625), (8.0, 2.925, 2.925), (16.0, 5.85, 5.85), (32.0, 11.7, 11.7), (64.0, 23.4, 23.4), (128.0, 46.8, 46.8), (256.0, 93.6, 93.6)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['ch:0', 'ch:1']
CHANNEL_COLORS: [16711680, 255]
DTYPE: uint16
PLANE_FILES: None
AXES: tczyx


In [ ]:
dataset.data[0]

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# It reads only the visualizing slice. Faster to open but not interactive.
# NOT RECOMMENDED FOR LARGE h5 DATASETS!
# To visualize lower resolution, use the next cell.
# To visualize the whole dataset at full resolution, convert data into ome-zarr

# start = time.time()
# viewer = dataset.visualize()
# print(f"{time.time()-start}")

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# The start_level-stop_level options, generate lower resolution layers and load them into memory
# NOTE: This works because the dataset has 5 resolution layers
# It takes some time because it has to open the files and read the corresponding layers
# Best would be to resave the dataset in ome-zarr (see next cell on how to do it)

start = time.time()
viewer = dataset.visualize( 
                        start_level = 5, 
                        stop_level = 6, 
                        in_memory = True 
                    )
print(f"{time.time()-start}")

In [ ]:
# --- Build pyramid if not already ---
dataset.build_pyramid(
                    num_levels=5, 
                    downscale_factor=2,
                    start_level=0
                    )

# --- Optional: Write to OME-Zarr format ---
output_zarr_path = f"{main_path}/{dataset_name}.zarr"
dataset.write(output_zarr_path)

In [ ]:
# # --- Show metadata summary for updated dataset ---
# print("Axes:", dataset.metadata["axes"])
# print("Size:", dataset.metadata["size"])
# print("Scales:", dataset.metadata["scales"])
# print("Chunk size:", dataset.chunks)
# print("Channels:", dataset.metadata["channel_names"])
# print("Colors:", dataset.metadata["channel_colors"])
# print("\n")
# for i in dataset.metadata:
#     print(f"{i.upper()}: {dataset.metadata[i]}")

# Now we use the ZarrManager to load the new data and display it

In [6]:
input_zarr_path = f"{main_path}/{dataset_name}.zarr"

In [7]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: (1,1,16,256,256)
# Here, we use the same chunks.
zarr_dataset = mm.ZarrManager(path=input_zarr_path, chunks=(1,1,32,2048,2048))

In [8]:
# --- Show metadata summary ---
print("Axes:", zarr_dataset.metadata["axes"])
print("Size:", zarr_dataset.metadata["size"])
print("Scales:", zarr_dataset.metadata["scales"])
print("Chunk size:", zarr_dataset.chunks)
print("Channels:", zarr_dataset.metadata["channel_names"])
print("Colors:", zarr_dataset.metadata["channel_colors"])
print("\n")
for i in zarr_dataset.metadata:
    print(f"{i.upper()}: {zarr_dataset.metadata[i]}")

Axes: tczyx
Size: [(1, 2, 4982, 16304, 12055), (1, 2, 2491, 8152, 6028), (1, 2, 1246, 4076, 3014), (1, 2, 623, 2038, 1507), (1, 2, 312, 1019, 754)]
Scales: [(4.0, 1.4625, 1.4625), (8.0, 2.925, 2.925), (16.0, 5.85, 5.85), (32.0, 11.7, 11.7), (64.0, 23.4, 23.4)]
Chunk size: (1, 1, 32, 2048, 2048)
Channels: ['ch:0', 'ch:1']
Colors: [16711680, 255]


SIZE: [(1, 2, 4982, 16304, 12055), (1, 2, 2491, 8152, 6028), (1, 2, 1246, 4076, 3014), (1, 2, 623, 2038, 1507), (1, 2, 312, 1019, 754)]
SCALES: [(4.0, 1.4625, 1.4625), (8.0, 2.925, 2.925), (16.0, 5.85, 5.85), (32.0, 11.7, 11.7), (64.0, 23.4, 23.4)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['ch:0', 'ch:1']
CHANNEL_COLORS: [16711680, 255]
DTYPE: uint16
PLANE_FILES: None
AXES: tczyx


In [ ]:
zarr_dataset.data[0]

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# It reads only the visualizing slice. Faster but not interactive.
start = time.time()
viewer = zarr_dataset.visualize()
print(f"{time.time()-start}")

In [ ]:
start = time.time()
viewer = zarr_dataset.visualize(                        
                        start_level = 2, 
                        stop_level = 3, 
                        in_memory = True 
)
print(f"{time.time()-start}")